In [8]:
from typing import Any
from scipy.interpolate import interp1d
import numpy as np
import copy
from src.util.enumerations import get_economic_scenario

ModuleNotFoundError: No module named 'src'

In [9]:
class MatterDefaults:
    def __init__(self):
        self.physical_use_ratio = 0.74 #per year
        self.discard_rate = 0.013 # per year
        self.conversion_rate_material_reserves = 0.0015 #per year
        self.recycling_rate = 0.2 # per year

In [10]:
class EmissionsAvoidedDefaults:
    # Constants for emissions calculations
    PROPORTION_PAPER = 0.15
    PROPORTION_PLASTIC = 0.10
    EFACTOR_VG_PAPER = 1304  # kg CO2/ton
    EFACTOR_REC_PAPER = -1300  # kg CO2/ton
    EFACTOR_VG_PLASTIC = 2400  # kg CO2/ton for PE
    EFACTOR_REC_PLASTIC = 163  # kg CO2/ton for PE

    # Constants for energy and fuel calculations
    EMISSION_FACTOR_DIESEL = 2.66  # kg CO2eq/Liter
    GENERATOR_EFFICIENCY = 0.33
    LOWER_HEATING_VALUE = 35.8  # MJ/Liter
    ENERGY_FACTOR_VG_PAPER = 36.85  # GJ/ton
    ENERGY_FACTOR_REC_PAPER = 26.21  # GJ/ton
    ENERGY_FACTOR_VG_PLASTIC = 93.46  # GJ/ton
    ENERGY_FACTOR_REC_PLASTIC = 33.01  # GJ/ton
    GLOBAL_WARMING_POTENTIAL_CO2 = 1

In [11]:
class MatterUse:
    """
    This class describes the matter-use dynamics in the JUSTICE model.
    """
    def __init__(
        self, 
        input_dataset,
        time_horizon,
        scenario,
        climate_ensembles,
        recycling_rate=None
        ):

        # Initialize defaults
        defaults = MatterDefaults()

        # Parameters
        self.physical_use_ratio = defaults.physical_use_ratio
        self.discard_rate = defaults.discard_rate
        self.conversion_rate_material_reserves = defaults.conversion_rate_material_reserves

        # Policy Lever
        self.recycling_rate = recycling_rate if recycling_rate is not None else defaults.recycling_rate

        # Saving the climate ensembles ?
        self.NUM_OF_ENSEMBLES = climate_ensembles
        
        # Saving the scenario
        self.scenario = get_economic_scenario(scenario)
        
        
        self.region_list = input_dataset.REGION_LIST
        self.material_intensity_array = copy.deepcopy(input_dataset.MATERIAL_INTENSITY_ARRAY)
        
        self.timestep = time_horizon.timestep
        self.data_timestep = time_horizon.data_timestep
        self.data_time_horizon = time_horizon.data_time_horizon
        self.model_time_horizon = time_horizon.model_time_horizon
        
        # Selecting only the required scenario
        self.material_intensity_array = self.material_intensity_array[:, :, self.scenario]


        if self.timestep != self.data_timestep:
            # Interpolate Material Intensity Dictionary
            self._interpolate_material_intensity() 
        
        """
        Initialize matter-use variables arrays
        """
        
        #TODO use neoclasical.py net_output
        self.net_output = net_output

        #Intializing the material intensity array Unit: kg/USD per year
        self.material_intensity = self.material_intensity_array

        #Intializing the material intensity array Unit: Gt per year
        self.material_consumption = np.zeros(
            (len(self.region_list), len(self.model_time_horizon), self.NUM_OF_ENSEMBLES)
        )

        #Intializing the in-use stock array Unit: Gt per year
        #TODO check this
        self.in_use_stock = copy.deepcopy(input_dataset.IN_USE_STOCK_INIT_ARRAY)

        #Intializing the discarded material array Unit: Gt per year
        self.discarded_material = np.zeros(
            (len(self.region_list), len(self.model_time_horizon), self.NUM_OF_ENSEMBLES)
        )

        #Intializing the recycled material array Unit: Gt per year
        self.recycled_material = np.zeros(
            (len(self.region_list), len(self.model_time_horizon), self.NUM_OF_ENSEMBLES)
        )

        #Intializing the waste array Unit: Gt per year
        self.waste = np.zeros(
            (len(self.region_list), len(self.model_time_horizon), self.NUM_OF_ENSEMBLES)
        )

        #Intializing the extracted matter array Unit: Gt per year
        self.extracted_matter = np.zeros(
            (len(self.region_list), len(self.model_time_horizon), self.NUM_OF_ENSEMBLES)
        )

        #Intializing the material reserves array Unit: Gt per year
        self.material_reserves = copy.deepcopy(input_dataset.MATERIAL_RESERVES_INIT_ARRAY)

        #Intializing the converted material reserves array Unit: Gt per year
        self.converted_material_reserves = np.zeros(
            (len(self.region_list), len(self.model_time_horizon), self.NUM_OF_ENSEMBLES)
        )

        #Intializing the material resources array Unit: Gt per year
        self.material_resources = copy.deepcopy(input_dataset.MATERIAL_RESOURCES_INIT_ARRAY)

        #Intializing the depletion ratio
        self.depletion_ratio = np.zeros(
            (len(self.region_list), len(self.model_time_horizon), self.NUM_OF_ENSEMBLES)
        )
    
    def run (self, timestep, recycling_rate):
        if len(recycling_rate.shape) == 1:
           recycling_rate = recycling_rate.reshape(-1, 1)
        self.get_material_consumption(timestep)
        self.get_in_use_stock(timestep)
        self.get_discarded_material(timestep)
        self.get_recycled_material(timestep)
        self.get_waste(timestep)
        self.get_extracted_matter(timestep)
        self.get_material_reserves(timestep)
        self.get_converted_material_reserves(timestep)
        self.get_material_resources(timestep)
        self.get_depletion_ratio(timestep)
           


    """
    Matter-use variable calculations functions
    """
    def get_material_consumption(self, timestep):
        # Calculate the domestic material consumption based on material intensity and net economic output
        self.material_consumption[:, timestep, :] = (
            self.material_intensity[:, timestep, :] * self.net_output[:, timestep, :] * 1000
        ) / 1_000_000_000  # Convert to Gt

    def get_in_use_stock(self, timestep):
        if timestep == 0:
            self.in_use_stock[:, timestep, :] = self.in_use_stock[:, timestep, :]
        else:
            self.in_use_stock[:, timestep, :] = (
                self.in_use_stock[:, timestep - 1, :]
                + self.material_consumption[:, timestep, :] * self.physical_use_ratio
                - self.discarded_material[:, timestep, :]
            )

    def get_discarded_material(self, timestep):
        self.discarded_material[:, timestep, :] = (
            self.discard_rate * self.in_use_stock[:, timestep - 1, :]
        )

    def get_recycled_material(self, timestep):
        self.recycled_material[:, timestep, :] = (
            self.recycling_rate * self.discarded_material[:, timestep, :]
        )

    def get_waste(self, timestep):
        self.waste[:, timestep, :] = (
            self.discarded_material[:, timestep, :] - self.recycled_material[:, timestep, :]
        )

    def get_extracted_matter(self, timestep):
        self.extracted_matter[:, timestep, :] = (
            self.material_consumption[:, timestep, :] - self.recycled_material[:, timestep, :]
        )

    def get_material_reserves(self, timestep):
        if timestep == 0:
            self.material_reserves[:, timestep, :] = self.material_reserves[:, timestep, :]
        else:
            self.material_reserves[:, timestep, :] = (
                self.material_reserves[:, timestep - 1, :]
                + self.converted_material_reserves[:, timestep, :]
                - self.extracted_matter[:, timestep, :]
            )

    def get_converted_material_reserves(self, timestep):
        self.converted_material_reserves[:, timestep, :] = (
            self.conversion_rate_material_reserves * self.material_resources[:, timestep - 1, :]
        )

    def get_material_resources(self, timestep):
        if timestep == 0:
            self.material_resources[:, timestep, :] = self.material_resources[:, timestep, :]
        else:
            self.material_resources[:, timestep, :] = (
                self.material_resources[:, timestep - 1, :]
                - self.converted_material_reserves[:, timestep, :]
            )

    def get_depletion_ratio(self, timestep):
        self.depletion_ratio[:, timestep, :] = (
            self.extracted_matter[:, timestep, :] / self.material_resources[:, timestep - 1, :]
        )
    
    
    """
    __________________________________________________________
    Emissions avoided through recycling of paper and plastics
    __________________________________________________________
    
    """
    def get_emissions_avoided(self, timestep):
        # Calculate proportions of recycled materials in gigatons (Gt)
        recycled_paper = self.recycled_material[:, timestep, :] * EmissionsAvoidedDefaults.PROPORTION_PAPER
        recycled_plastic = self.recycled_material[:, timestep, :] * EmissionsAvoidedDefaults.PROPORTION_PLASTIC

        # Calculate GHG emissions avoided
        em_ghg_avoided = self.calculate_emissions_avoided(recycled_paper, recycled_plastic)

        # Calculate energy savings
        e_total_saved = self.calculate_energy_saved(recycled_paper, recycled_plastic)

        # Calculate fuel saved and CO2 emissions avoided
        em_co2_avoided = self.calculate_co2_avoided(e_total_saved)

        # Total emissions avoided
        em_total = em_ghg_avoided + em_co2_avoided
        self.emissions_avoided[:, timestep, :] = em_total

        # TODO not implementedAdjust Carbon Intensity
        # self.adjust_carbon_intensity(timestep, em_total) ?

    def calculate_emissions_avoided(self, recycled_paper, recycled_plastic):
        em_ghg_vg_paper = EmissionsAvoidedDefaults.EFACTOR_VG_PAPER * recycled_paper * EmissionsAvoidedDefaults.GLOBAL_WARMING_POTENTIAL_CO2
        em_ghg_rec_paper = EmissionsAvoidedDefaults.EFACTOR_REC_PAPER * recycled_paper * EmissionsAvoidedDefaults.GLOBAL_WARMING_POTENTIAL_CO2
        em_ghg_vg_plastic = EmissionsAvoidedDefaults.EFACTOR_VG_PLASTIC * recycled_plastic * EmissionsAvoidedDefaults.GLOBAL_WARMING_POTENTIAL_CO2
        em_ghg_rec_plastic = EmissionsAvoidedDefaults.EFACTOR_REC_PLASTIC * recycled_plastic * EmissionsAvoidedDefaults.GLOBAL_WARMING_POTENTIAL_CO2

        em_ghg_avoided_paper = em_ghg_vg_paper - em_ghg_rec_paper
        em_ghg_avoided_plastic = em_ghg_vg_plastic - em_ghg_rec_plastic

        return em_ghg_avoided_paper + em_ghg_avoided_plastic

    def calculate_energy_saved(self, recycled_paper, recycled_plastic):
        e_vg_paper = recycled_paper * EmissionsAvoidedDefaults.ENERGY_FACTOR_VG_PAPER
        e_rec_paper = recycled_paper * EmissionsAvoidedDefaults.ENERGY_FACTOR_REC_PAPER
        e_vg_plastic = recycled_plastic * EmissionsAvoidedDefaults.ENERGY_FACTOR_VG_PLASTIC
        e_rec_plastic = recycled_plastic * EmissionsAvoidedDefaults.ENERGY_FACTOR_REC_PLASTIC

        e_total_saved_paper = e_vg_paper - e_rec_paper
        e_total_saved_plastic = e_vg_plastic - e_rec_plastic

        return e_total_saved_paper + e_total_saved_plastic

    def calculate_co2_avoided(self, e_total_saved):
        fuel_saved = (e_total_saved * 100 * 41.868) / (EmissionsAvoidedDefaults.GENERATOR_EFFICIENCY * EmissionsAvoidedDefaults.LOWER_HEATING_VALUE)  # Liters/day
        return fuel_saved * EmissionsAvoidedDefaults.EMISSION_FACTOR_DIESEL

    #TODO check ALL THIS BELOW
    def _interpolate_material_intensity(self): 
        interp_data = np.zeros(
            (
                self.material_intensity_array.shape[0],
                len(self.model_time_horizon),
                # self.gdp_array.shape[2],
            )
        )
        for i in range(self.material_intensity_array.shape[0]):
            f = interp1d(
                self.data_time_horizon, self.material_intensity_array[i, :], kind="linear"  # , j
            )
            interp_data[i, :] = f(self.model_time_horizon)  # , j

        self.material_intensity_array= interp_data
    
    def __getattribute__(self, __name: str) -> Any:
        """
        This method returns the value of the attribute of the class.
        """
        return object.__getattribute__(self, __name)